<a href="https://colab.research.google.com/github/Fariha-Asif/scien/blob/main/scientific_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Install necessary packages (only if not already installed)
!pip install ipywidgets
!pip install plotly

# Import libraries
import plotly.graph_objs as go
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# Set Plotly renderer for Google Colab
pio.renderers.default = 'colab'

# Display for the input and output
input_box = widgets.Text(
    value='',
    placeholder='Enter expression',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

output_box = widgets.Output()

# Define button styles using predefined styles
button_style_mapping = {
    'sin': 'info',
    'cos': 'info',
    'tan': 'info',
    'sqrt': 'warning',
    'log': 'warning',
    'pi': 'warning',
    'x': '',
    'plot': 'danger',
    'clear': 'danger',
    '=': 'success',
    'exp': 'info',
    'abs': 'info',
    '+': ''  # Default style for '+' button
}

# Function to create a button with specific label and style
def create_button(label):
    style = {'button_style': button_style_mapping.get(label, '')}
    return widgets.Button(description=label, style=style, layout=widgets.Layout(width='60px', height='40px'))

# Define buttons with all necessary labels, including '+'
buttons = [
    ['7', '8', '9', '/', 'sin'],
    ['4', '5', '6', '*', 'cos'],
    ['1', '2', '3', '-', 'tan'],
    ['0', '.', '(', ')', 'sqrt'],
    ['pi', 'x', '^', 'log', 'plot'],
    ['exp', 'abs', '+', '=', 'clear']
]

# Create a grid of buttons
button_grid = []
for row in buttons:
    button_row = []
    for label in row:
        button = create_button(label)
        button_row.append(button)
    button_grid.append(widgets.HBox(button_row))

# Define a safe list of functions and constants using NumPy
allowed_names = {
    'sin': np.sin,
    'cos': np.cos,
    'tan': np.tan,
    'sqrt': np.sqrt,
    'log': np.log,
    'exp': np.exp,
    'abs': np.abs,
    'pi': np.pi,
    'e': np.e,
    'x': None  # Placeholder for the variable 'x' during plotting
}

def sanitize_expression(expression):
    """
    Replace Unicode minus signs with standard hyphen-minus.
    """
    return expression.replace('−', '-')

def eval_expression(expression, x_value=None):
    """
    Safely evaluate the mathematical expression using allowed names.
    If x_value is provided, include it in the evaluation context.
    """
    # Sanitize the expression
    expression = sanitize_expression(expression)

    local_dict = allowed_names.copy()
    if x_value is not None:
        local_dict['x'] = x_value
    return eval(expression, {"__builtins__": {}}, local_dict)

def plot_function(expression):
    """
    Plot the mathematical function using Plotly.
    """
    # Sanitize the expression
    expression = sanitize_expression(expression)

    # Create a range of x values
    x = np.linspace(-10, 10, 400)

    try:
        # Evaluate y using the expression with x as a NumPy array
        y = eval_expression(expression, x_value=x)
    except Exception as e:
        with output_box:
            clear_output()
            print(f"Error in expression: {e}")
        return

    # Check if y is a valid array
    if not isinstance(y, (list, np.ndarray)):
        with output_box:
            clear_output()
            print("Error: The expression must be a function of 'x'.")
        return

    # Check if y has finite values
    if not np.all(np.isfinite(y)):
        with output_box:
            clear_output()
            print("Error: The expression resulted in non-finite values (NaN or Inf).")
        return

    # Create the plot using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=expression))
    fig.update_layout(
        title=f'Plot of {expression}',
        xaxis_title='x',
        yaxis_title='y'
    )

    with output_box:
        clear_output()
        # Display the plot using the default renderer
        fig.show()

# Function to handle button clicks
def on_button_click(b):
    if b.description == 'clear':
        input_box.value = ''
        with output_box:
            clear_output()
    elif b.description == '=':
        try:
            # Replace caret (^) with exponentiation operator (**)
            expression = input_box.value.replace('^', '**')
            # Evaluate the expression safely
            result = eval_expression(expression)
            with output_box:
                clear_output()
                print("Result:", result)
        except ZeroDivisionError:
            with output_box:
                clear_output()
                print("Math Error: Division by zero is undefined.")
        except SyntaxError:
            with output_box:
                clear_output()
                print("Syntax Error: Please check the expression for correct syntax.")
        except NameError:
            with output_box:
                clear_output()
                print("Name Error: Ensure all functions and variables are correctly spelled.")
        except Exception as e:
            with output_box:
                clear_output()
                print("Error:", e)
    elif b.description == 'plot':
        try:
            expression = input_box.value.replace('^', '**')
            plot_function(expression)
        except Exception as e:
            with output_box:
                clear_output()
                print("Error:", e)
    else:
        # Append the button label to the input box
        if b.description == 'pi':
            input_box.value += str(np.pi)
        elif b.description in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'abs']:
            input_box.value += b.description + '('
        elif b.description == '^':
            input_box.value += '**'
        elif b.description == 'x':
            input_box.value += 'x'
        elif b.description == '+':
            input_box.value += '+'
        else:
            input_box.value += b.description

# Attach the click event to all buttons
for row in button_grid:
    for button in row.children:
        button.on_click(on_button_click)

# Arrange everything in a vertical layout
calculator = widgets.VBox([
    input_box,
    widgets.VBox(button_grid),
    output_box
])

# Display the calculator
display(calculator)


In [52]:
import numpy as np
print(np.__version__)


1.26.4


In [65]:
# app.py

import streamlit as st
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Set Plotly renderer for Streamlit
pio.renderers.default = "iframe_connected"

# Title of the App
st.title("📟 Scientific Calculator with Plotting")

# Initialize Session State for expression and history
if 'expression' not in st.session_state:
    st.session_state.expression = ""
if 'history' not in st.session_state:
    st.session_state.history = []

# Define allowed names for safe evaluation
allowed_names = {
    'sin': np.sin,
    'cos': np.cos,
    'tan': np.tan,
    'sqrt': np.sqrt,
    'log': np.log,
    'exp': np.exp,
    'abs': np.abs,
    'pi': np.pi,
    'e': np.e,
    'factorial': lambda x: np.math.factorial(int(x)),
    'asin': np.arcsin,
    'acos': np.arccos,
    'atan': np.arctan,
    'floor': np.floor,
    'ceil': np.ceil,
    'round': np.round,
    'x': None  # Placeholder for plotting
}

def sanitize_expression(expression):
    """
    Replace Unicode minus signs with standard hyphen-minus.
    """
    return expression.replace('−', '-')

def eval_expression(expression, x_value=None):
    """
    Safely evaluate the mathematical expression using allowed names.
    If x_value is provided, include it in the evaluation context.
    """
    # Sanitize the expression
    expression = sanitize_expression(expression)

    # Update 'x' in allowed_names if plotting
    local_dict = allowed_names.copy()
    if x_value is not None:
        local_dict['x'] = x_value

    try:
        return eval(expression, {"__builtins__": {}}, local_dict)
    except Exception as e:
        raise e

def plot_function(expression):
    """
    Plot the mathematical function using Plotly.
    """
    # Sanitize the expression
    expression = sanitize_expression(expression)

    # Create a range of x values
    x = np.linspace(-10, 10, 400)

    try:
        # Evaluate y using the expression with x as a NumPy array
        y = eval_expression(expression, x_value=x)
    except Exception as e:
        st.error(f"Error in expression: {e}")
        return

    # Check if y is a valid array
    if not isinstance(y, (list, np.ndarray)):
        st.error("Error: The expression must be a function of 'x'.")
        return

    # Check if y has finite values
    if not np.all(np.isfinite(y)):
        st.error("Error: The expression resulted in non-finite values (NaN or Inf).")
        return

    # Create the plot using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=expression))
    fig.update_layout(
        title=f'Plot of {expression}',
        xaxis_title='x',
        yaxis_title='y',
        template='plotly_dark'
    )

    st.plotly_chart(fig, use_container_width=True)

def add_to_expression(value):
    """
    Add a value to the current expression.
    """
    st.session_state.expression += value

def clear_expression():
    """
    Clear the current expression.
    """
    st.session_state.expression = ""

def calculate_expression():
    """
    Evaluate the current expression and display the result.
    """
    expression = st.session_state.expression
    if not expression:
        st.warning("Please enter an expression to evaluate.")
        return
    try:
        result = eval_expression(expression)
        st.session_state.history.append(f"{expression} = {result}")
        st.success(f"Result: {result}")
    except ZeroDivisionError:
        st.error("Math Error: Division by zero is undefined.")
    except SyntaxError:
        st.error("Syntax Error: Please check the expression for correct syntax.")
    except NameError:
        st.error("Name Error: Ensure all functions and variables are correctly spelled.")
    except Exception as e:
        st.error(f"Error: {e}")

def plot_expression():
    """
    Plot the current expression.
    """
    expression = st.session_state.expression
    if not expression:
        st.warning("Please enter an expression to plot.")
        return
    try:
        plot_function(expression)
    except Exception as e:
        st.error(f"Error: {e}")

# Display the current expression
st.text_input("Expression:", st.session_state.expression, key='expr_input', disabled=True)

# Calculator Buttons Layout
button_labels = [
    ['7', '8', '9', '/', 'sin'],
    ['4', '5', '6', '*', 'cos'],
    ['1', '2', '3', '-', 'tan'],
    ['0', '.', '(', ')', 'sqrt'],
    ['pi', 'x', '^', 'log', 'plot'],
    ['exp', 'abs', '+', '=', 'clear']
]

# Create buttons in a grid layout
for row in button_labels:
    cols = st.columns(len(row))
    for idx, label in enumerate(row):
        btn = cols[idx].button(label)
        if btn:
            if label == '=':
                calculate_expression()
            elif label == 'plot':
                plot_expression()
            elif label == 'clear':
                clear_expression()
                st.success("Cleared the expression.")
            elif label == 'pi':
                add_to_expression(str(np.pi))
            elif label in ['sin', 'cos', 'tan', 'sqrt', 'log', 'exp', 'abs']:
                add_to_expression(label + '(')
            elif label == '^':
                add_to_expression('**')
            else:
                add_to_expression(label)

# Display History
if st.session_state.history:
    st.subheader("📝 Calculation History")
    for entry in reversed(st.session_state.history[-10:]):
        st.write(entry)

2024-10-16 17:09:36.148 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 17:09:36.318 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-16 17:09:36.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 17:09:36.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 17:09:36.326 Session state does not function when running a script without `streamlit run`
2024-10-16 17:09:36.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 17:09:36.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 17:09:36.331 Thread 'MainThread': missing ScriptRunContext! 

In [62]:
!pip install plotly


In [64]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
